In [3]:
# !pip install torch==2.1.1

# 1.1 数据类型
学习用于存储机器学习模型参数的常见数据类型。

![不同数据类型的二进制表示过程](./img/1_binary_data_storage.png)

<!-- <img src="./img/1_binary_data_storage.png" alt="不同数据类型的二进制表示过程" width="800" /> -->




图片展示了计算机是如何使用二进制进行数据的存储，以32为浮点数（FP32）为例，计算机使用了32位二进制进行表示，其中符号位占了1位（首位），指数位占了8位，其余23位为小数位。

对于一个10B的模型使用FP64进行存储的模型来说，其占用的静态存储空间约为74.5GB，因为每个参数占64/8个字节，10亿参数则占用了80亿字节，即74.5GB。

In [5]:
import torch

In [ ]:
# 使用finfo查看torch中浮点数的数据情况
torch.finfo(torch.float32)

finfo(resolution=1e-06, min=-3.40282e+38, max=3.40282e+38, eps=1.19209e-07, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=float32)

In [ ]:
# bf16和pf32有一样的指数位宽，因此有相同的动态范围，但是bf16的精度（小数位的位宽）没有fp32多
torch.finfo(torch.bfloat16)

finfo(resolution=0.01, min=-3.38953e+38, max=3.38953e+38, eps=0.0078125, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=bfloat16)

In [ ]:
# fp16的动态范围比bf16小，但是精度比bf16高（能表示更精确的浮点数）
torch.finfo(torch.float16)

finfo(resolution=0.001, min=-65504, max=65504, eps=0.000976562, smallest_normal=6.10352e-05, tiny=6.10352e-05, dtype=float16)

In [14]:
# 使用iinfo查看int类型的情况
torch.iinfo(torch.int8)

iinfo(min=-128, max=127, dtype=int8)

In [15]:
# 无符号数
torch.iinfo(torch.uint8)

iinfo(min=0, max=255, dtype=uint8)

In [ ]:
# torch中没有int4和int2的原生实现，因此会报错。
# torch.iinfo(torch.int4)

AttributeError: module 'torch' has no attribute 'int4'

In [17]:
# 定义一个fp64位的小数
v = 1/3

In [23]:
# 展示前60位
format(v,'.60f')

'0.333333333333333314829616256247390992939472198486328125000000'

In [24]:
# 使用tensor存储64位浮点数
tensor_fp64 = torch.tensor(v, dtype = torch.float64)
print(f"fp64 tensor: {format(tensor_fp64.item(), '.60f')}")

fp64 tensor: 0.333333333333333314829616256247390992939472198486328125000000


In [25]:
# 分别转化为fp32\fp16\bf16
tensor_fp32 = torch.tensor(v, dtype = torch.float32)
tensor_fp16 = torch.tensor(v, dtype = torch.float16)
tensor_bf16 = torch.tensor(v, dtype = torch.bfloat16)

In [26]:
print(f"fp64 tensor: {format(tensor_fp64.item(), '.60f')}")
print(f"fp32 tensor: {format(tensor_fp32.item(), '.60f')}")
print(f"fp16 tensor: {format(tensor_fp16.item(), '.60f')}")
print(f"bf16 tensor: {format(tensor_bf16.item(), '.60f')}")

fp64 tensor: 0.333333333333333314829616256247390992939472198486328125000000
fp32 tensor: 0.333333343267440795898437500000000000000000000000000000000000
fp16 tensor: 0.333251953125000000000000000000000000000000000000000000000000
bf16 tensor: 0.333984375000000000000000000000000000000000000000000000000000


# 1.2 量化的本质

<img src="./img/1_essence.png" alt="数据类型映射关系" width="800" />

量化实际上就是集合之间的映射，需要找个一个映射方式尽量减少损失的情况下使用更小的集合表达更大的集合。